In [1]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd
import random
import requests
import rasterio
import itertools
import matplotlib.image
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

from pathlib import Path
from rasterio.windows import Window


In [2]:
# lifting contents from https://github.com/eu-cdse/notebook-samples/blob/main/geo/odata_basics.ipynb atm
catalogue_odata_url = "https://catalogue.dataspace.copernicus.eu/odata/v1"

collection_name = "SENTINEL-2"
product_type = "S2MSI1C"
max_cloud_cover = 1
#corners = list(itertools.product(["20.888443", "21.124649"], ["52.169721", "52.271099"]))
corners = list(itertools.product(["23.0", "24.5"], ["56.5", "57.2"]))
poly = [" ".join(corners[i]) for i in [0, 2, 3, 1, 0]]
aoi = f"POLYGON(({",".join(poly)}))"
search_period_start = "2023-06-01T00:00:00.000Z"
search_period_end = "2023-06-10T00:00:00.000Z"


In [3]:
search_query = f"{catalogue_odata_url}/Products?$filter=Collection/Name eq '{collection_name}' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{product_type}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and ContentDate/Start gt {search_period_start} and ContentDate/Start lt {search_period_end}"

print(f"""\n{search_query.replace(' ', "%20")}\n""")



https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name%20eq%20'SENTINEL-2'%20and%20Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20'productType'%20and%20att/OData.CSC.StringAttribute/Value%20eq%20'S2MSI1C')%20and%20OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((23.0%2056.5,24.5%2056.5,24.5%2057.2,23.0%2057.2,23.0%2056.5))')%20and%20ContentDate/Start%20gt%202023-06-01T00:00:00.000Z%20and%20ContentDate/Start%20lt%202023-06-10T00:00:00.000Z



In [4]:
# collection rqs don't require you to auth?
response = requests.get(search_query).json()
result = pd.DataFrame.from_dict(response["value"])

result.head(3)


,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,9bb0f268-aecc-44eb-8139-1d940d5c1363,S2B_MSIL1C_20230605T092559_N0509_R136_T35VLC_2...,application/octet-stream,622567953,2023-06-05T12:27:09.024000Z,2023-06-05T12:40:55.580371Z,2023-06-05T12:44:28.320715Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/05/S2B_MSIL...,"[{'Value': '784ef5a63fb2bea348d7e2a7afce5c9e',...","{'Start': '2023-06-05T09:25:59.024000Z', 'End'...",geography'SRID=4326;POLYGON ((24.0584093747756...,"{'type': 'Polygon', 'coordinates': [[[24.05840..."
1,application/octet-stream,b920599a-fd15-4dbc-85d5-5388fcadbb72,S2B_MSIL1C_20230605T092559_N0509_R136_T34VFH_2...,application/octet-stream,74876718,2023-06-05T12:47:20.854000Z,2023-06-05T12:56:29.926675Z,2023-06-05T13:05:55.095736Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/05/S2B_MSIL...,"[{'Value': '38facb25ad41f92d5a29629023b61524',...","{'Start': '2023-06-05T09:25:59.024000Z', 'End'...",geography'SRID=4326;POLYGON ((24.0569254780051...,"{'type': 'Polygon', 'coordinates': [[[24.05692..."
2,application/octet-stream,78fb789b-9333-4ed4-806d-d1bda8c09c76,S2A_MSIL1C_20230606T095031_N0509_R079_T35VLD_2...,application/octet-stream,489635087,2023-06-06T14:31:30.247000Z,2023-06-06T14:39:05.049206Z,2023-06-07T01:04:11.101872Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/06/S2A_MSIL...,"[{'Value': '315241c699e521a122ccd02760a068fe',...","{'Start': '2023-06-06T09:50:31.025000Z', 'End'...",geography'SRID=4326;POLYGON ((25.2322533597395...,"{'type': 'Polygon', 'coordinates': [[[25.23225..."


In [5]:
# filter by cloud coverage
search_query = f"{search_query} and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover})"
print(f"""\n{search_query.replace(' ', "%20")}\n""")

response = requests.get(search_query).json()
result = pd.DataFrame.from_dict(response["value"])

result.head(3)



https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name%20eq%20'SENTINEL-2'%20and%20Attributes/OData.CSC.StringAttribute/any(att:att/Name%20eq%20'productType'%20and%20att/OData.CSC.StringAttribute/Value%20eq%20'S2MSI1C')%20and%20OData.CSC.Intersects(area=geography'SRID=4326;POLYGON((23.0%2056.5,24.5%2056.5,24.5%2057.2,23.0%2057.2,23.0%2056.5))')%20and%20ContentDate/Start%20gt%202023-06-01T00:00:00.000Z%20and%20ContentDate/Start%20lt%202023-06-10T00:00:00.000Z%20and%20Attributes/OData.CSC.DoubleAttribute/any(att:att/Name%20eq%20'cloudCover'%20and%20att/OData.CSC.DoubleAttribute/Value%20le%201)



,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,b4f81881-85b7-4925-af64-09ad9ea62764,S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_2...,application/octet-stream,779589021,2023-06-08T12:53:41.289000Z,2023-06-08T13:02:18.840487Z,2023-06-22T23:35:31.032869Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/08/S2B_MSIL...,"[{'Value': 'a2150e8f82b8e8f1d36fb3e0db7430c7',...","{'Start': '2023-06-08T09:35:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((23.6438298900325...,"{'type': 'Polygon', 'coordinates': [[[23.64382..."
1,application/octet-stream,a04b890f-bc50-430b-9a51-41e42ba3a741,S2A_MSIL1C_20230606T095031_N0509_R079_T34VFH_2...,application/octet-stream,766678264,2023-06-06T15:01:10.529000Z,2023-06-06T15:13:06.892867Z,2023-06-10T03:50:51.138552Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/06/S2A_MSIL...,"[{'Value': '49daee3eb77421305752d93151247583',...","{'Start': '2023-06-06T09:50:31.025000Z', 'End'...",geography'SRID=4326;POLYGON ((24.4100100361209...,"{'type': 'Polygon', 'coordinates': [[[24.41001..."
2,application/octet-stream,4b1c6226-ed19-41ad-ace6-e2edef5a36c9,S2B_MSIL1C_20230608T093549_N0509_R036_T34VFH_2...,application/octet-stream,825281640,2023-06-08T12:47:02.564000Z,2023-06-08T12:58:54.685349Z,2023-06-22T23:31:42.231821Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L1C/2023/06/08/S2B_MSIL...,"[{'Value': 'ed999043e1f55074310ccefa1ed75c8b',...","{'Start': '2023-06-08T09:35:49.024000Z', 'End'...",geography'SRID=4326;POLYGON ((22.6389064284462...,"{'type': 'Polygon', 'coordinates': [[[22.63890..."


In [6]:
from settings import username, password

auth_server_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
data = {
    "client_id": "cdse-public",
    "grant_type": "password",
    "username": username,
    "password": password,
}

response = requests.post(auth_server_url, data=data, verify=True, allow_redirects=False)
access_token = json.loads(response.text)["access_token"]


In [7]:
product_identifier = result.iloc[0, 1]
product_name = result.iloc[0, 2]

session = requests.Session()
session.headers["Authorization"] = f"Bearer {access_token}"


In [8]:
url = f"{catalogue_odata_url}/Products({product_identifier})/Nodes({product_name})/Nodes(MTD_MSIL1C.xml)/$value"

print(url)

response = session.get(url, allow_redirects=False)
while response.status_code in (301, 302, 303, 307):
    url = response.headers["Location"]
    response = session.get(url, allow_redirects=False)
    print(response)

file = session.get(url, verify=False, allow_redirects=True)

print(file)

outfile = Path.home() / "Projs/bulbulis/MTD_MSIL1C.xml"
outfile.write_bytes(file.content)


https://catalogue.dataspace.copernicus.eu/odata/v1/Products(b4f81881-85b7-4925-af64-09ad9ea62764)/Nodes(S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_20230608T102808.SAFE)/Nodes(MTD_MSIL1C.xml)/$value
<Response [200]>


/Users/kristaps/Projs/bulbulis/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.dataspace.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


45545

In [9]:
tree = ET.parse(str(outfile))
root = tree.getroot()

band_location = []
band_location.append(f"{product_name}/{root[0][0][12][0][0][1].text}.jp2".split("/"))
band_location.append(f"{product_name}/{root[0][0][12][0][0][2].text}.jp2".split("/"))
band_location.append(f"{product_name}/{root[0][0][12][0][0][3].text}.jp2".split("/"))


In [10]:
band_location

[['S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_20230608T102808.SAFE',
  'GRANULE',
  'L1C_T35VLD_A032666_20230608T093840',
  'IMG_DATA',
  'T35VLD_20230608T093549_B02.jp2'],
 ['S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_20230608T102808.SAFE',
  'GRANULE',
  'L1C_T35VLD_A032666_20230608T093840',
  'IMG_DATA',
  'T35VLD_20230608T093549_B03.jp2'],
 ['S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_20230608T102808.SAFE',
  'GRANULE',
  'L1C_T35VLD_A032666_20230608T093840',
  'IMG_DATA',
  'T35VLD_20230608T093549_B04.jp2']]

In [ ]:
# does this not cost credits?
bands = []
for band_file in band_location:
    url = f"{catalogue_odata_url}/Products({product_identifier})/Nodes({product_name})/Nodes({band_file[1]})/Nodes({band_file[2]})/Nodes({band_file[3]})/Nodes({band_file[4]})/$value"
    print(url)
    response = session.get(url, allow_redirects=False)
    while response.status_code in (301, 302, 303, 307):
        url = response.headers["Location"]
        response = session.get(url, allow_redirects=False)
        print(response)
    file = session.get(url, verify=False, allow_redirects=True)
    
    outfile = Path.home() / f"Projs/bulbulis/{band_file[4]}"
    outfile.write_bytes(file.content)
    bands.append(str(outfile))
    print("Saved:", band_file[4])
    

https://catalogue.dataspace.copernicus.eu/odata/v1/Products(b4f81881-85b7-4925-af64-09ad9ea62764)/Nodes(S2B_MSIL1C_20230608T093549_N0509_R036_T35VLD_20230608T102808.SAFE)/Nodes(GRANULE)/Nodes(L1C_T35VLD_A032666_20230608T093840)/Nodes(IMG_DATA)/Nodes(T35VLD_20230608T093549_B02.jp2)/$value
<Response [200]>


/Users/kristaps/Projs/bulbulis/.venv/lib/python3.13/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.dataspace.copernicus.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
%matplotlib inline

xsize, ysize = 1000, 1000
xoff, yoff, xmax, ymax = 0, 0, 0, 0
n = 2

for band_file in bands:
    full_band = rasterio.open(band_file, driver="JP2OpenJPEG")
    if xmax == 0:
        xmin, xmax = 0, full_band.width - xsize
    if ymax == 0:
        ymin, ymax = 0, full_band.height - ysize
    if xoff == 0:
        xoff, yoff = random.randint(xmin, xmax), random.randint(ymin, ymax)
    window = Window(xoff, yoff, xsize, ysize)
    transform = full_band.window_transform(window)
    profile = full_band.profile
    crs = full_band.crs
    profile.update({"height": xsize, "width": ysize, "transform": transform})
    with rasterio.open(
        f"{Path.home()}/Projs/bulbulis/patch_band_{n}.jp2", "w", **profile
    ) as patch_band:
        # Read the data from the window and write it to the output raster
        patch_band.write(full_band.read(window=window))
    print(f"Patch for band {n} created")
    n += 1


In [ ]:
# Read the patch files
band2 = rasterio.open(f"{Path.home()}/Projs/bulbulis/patch_band_2.jp2", driver="JP2OpenJPEG")  # blue
band3 = rasterio.open(f"{Path.home()}/Projs/bulbulis/patch_band_3.jp2", driver="JP2OpenJPEG")  # green
band4 = rasterio.open(f"{Path.home()}/Projs/bulbulis/patch_band_4.jp2", driver="JP2OpenJPEG")  # red

red = band4.read(1)
green = band3.read(1)
blue = band2.read(1)

# Normalize the pixel values and apply gain
gain = 2
red_n = np.clip(red * gain / 10000, 0, 1)
green_n = np.clip(green * gain / 10000, 0, 1)
blue_n = np.clip(blue * gain / 10000, 0, 1)

# Create composite image
rgb_composite_n = np.dstack((red_n, green_n, blue_n))

# Display image
plt.imshow(rgb_composite_n)

# Save image to file
matplotlib.image.imsave(f"{Path.home()}/Projs/bulbulis/Sentinel2_true_color.jpeg", rgb_composite_n)
print("Saved as:", outfile)
